In [3]:
from kvadrati import Kvadrat


ModuleNotFoundError: No module named 'sage'

In [2]:

#PRVA VARJANTA
p = MixedIntegerLinearProgram(maximization = False)
n = 3
KVADRATI_SEZ = []
x = []
y = []
z1 = p.new_variable #dolzina stranice velikega kvadrata
p.set_objective(z1) #zelim minimizirat , mi ILP ne poje 
for i in range(1,(n+1)):
    ljala = p.new_variable
    x.append(ljala)
    tra = p.new_variable
    y.append(tra)
for i in range(1,(n+1)):
    sez = [i,x[i-1],x[i-1]]
    KVADRATI_SEZ.append(sez)
#(dolzina, x,y)
for kvadrat1 in KVADRATI_SEZ: #x + dolzina daljice <= dolzina velikega kvadrata 
    p.add_constraint(kvadrat1[1] + kvadrat1[2]  - z1 <= 0) #Tule je problem varible + cifra - varible 
    p.add_constraint(kvadrat1[1] + kvadrat1[2] - z1 <= 0) #Tule je problem 
for kvadrat1 in KVADRATI_SEZ:
    for kvadrat2 in KVADRATI_SEZ:
        Log = p.new_variable(binary=True)
        Log = (kvadrat1[2] + kvadrat1[1] <= kvadrat2[2] or kvadrat1[3] + kvadrat1[1] <= kvadrat2[3] or kvadrat2[2] + kvadrat2[1] <= kvadrat1[2] or kvadrat2[3] + kvadrat2[1] <= kvadrat1[3] )
        p.add_constraint(Log == 1)

NameError: name 'MixedIntegerLinearProgram' is not defined